## Advent of Code

The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

In [ ]:
import sys
sys.path.append('..')
from utils import *

inp = get_data(day=6, year=2024)
samp = '''....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...'''
print(samp)

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.
Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

```
....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```
Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

```
....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
```
Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
```
This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):
```
....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
```
By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:
```
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
```
In this example, the guard will visit 41 distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?

# Plan

Parse the map.
- Represent the map (we'll call it a "grid") as a 2D array.
- Use an extra class (State) to represent guard location and direction.
- Track past states.

## Data structures

In [ ]:
samp.splitlines()

['....#.....',
 '.........#',
 '..........',
 '..#.......',
 '.......#..',
 '..........',
 '.#..^.....',
 '........#.',
 '#.........',
 '......#...']

In [ ]:
from enum import Enum

class Direction(Enum):
    UP = 0
    RIGHT = 1
    DOWN = 2
    LEFT = 3

In [ ]:
from fastcore.basics import store_attr, BasicRepr

class State(BasicRepr):
    def __init__(self, position, direction):
        store_attr()
    
    def __eq__(self, other):
        return self.position == other.position and self.direction == other.direction
    
    def __hash__(self):
        return hash((self.position, self.direction))

State((5, 5), Direction.UP)

State(position=(5, 5), direction=<Direction.UP: 0>)

In [ ]:
from fastcore.basics import store_attr, BasicRepr

class State(BasicRepr):
    def __init__(self, position, direction):
        store_attr()
    
    def __eq__(self, other):
        return self.position == other.position and self.direction == other.direction
    
    def __hash__(self):
        return hash((self.position, self.direction))

State((5, 5), Direction.LEFT)

State(position=(5, 5), direction=<Direction.LEFT: 3>)

## Parsing

In [ ]:
def parse_direction(c): return Direction(['^','>','v','<'].index(c))

parse_direction('>')

<Direction.RIGHT: 1>

In [ ]:
xs = samp.splitlines()
xs

['....#.....',
 '.........#',
 '..........',
 '..#.......',
 '.......#..',
 '..........',
 '.#..^.....',
 '........#.',
 '#.........',
 '......#...']

In [ ]:
for i, row in enumerate(xs):
    for j, c in enumerate(row):
        if c not in '.#':
            guard_pos = (i, j)
            guard_symbol = c
            break
guard_pos, guard_symbol

((6, 4), '^')

In [ ]:
def parse_init_state(xs):
    for i, row in enumerate(xs):
        for j, c in enumerate(row):
            if c not in '.#O':
                guard_pos = (i, j)
                guard_symbol = c
                break
    d = parse_direction(guard_symbol)
    return State(guard_pos, d)
init_state = parse_init_state(xs)
init_state

State(position=(6, 4), direction=<Direction.UP: 0>)

In [ ]:
x = xs[6]
x

'.#..^.....'

In [ ]:
L(list(x)).map(lambda c: c in "#")

(#10) [False,True,False,False,False,False,False,False,False,False]

In [ ]:
def parse_grid_line(x):
    return L(list(x)).map(lambda c: c in "#O")
parse_grid_line(x)

(#10) [False,True,False,False,False,False,False,False,False,False]

In [ ]:
import numpy as np

def parse_grid(xs):
    return np.array(L(xs).map(parse_grid_line))
grid = parse_grid(xs)
grid

array([[False, False, False, False,  True, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
         True],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False,  True, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False,  True,
        False],
       [ True, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False,  True, False, False,
        False]])

## Printing

In [ ]:
def show_direction_char(direction): return ['^','>','v','<'][direction.value]

show_direction_char(Direction.LEFT)

'<'

In [ ]:
def show_char(pos, grid, states, curr_state):
    i, j = pos
    c = '#' if grid[i,j] else '.'
    if (i,j) in L(states).map(lambda s: s.position): c = 'X'
    if (i,j) == curr_state.position: c = show_direction_char(curr_state.direction)
    return c
show_char((6, 4), grid, [], init_state)

'^'

In [ ]:
grid[6,]

array([False,  True, False, False, False, False, False, False, False,
       False])

In [ ]:
def show_row(row_num, grid, states, curr_state):
    positions = L.range(len(grid[0])).map(lambda o: (row_num, o))
    cs = positions.map(show_char, grid=grid, states=states, curr_state=curr_state)
    return ''.join(cs)
show_row(6, grid, [], init_state)

'.#..^.....'

In [ ]:
def show(grid, states, curr_state):
    rs = L.range(grid).map(show_row, grid=grid, states=states, curr_state=curr_state)
    return '\n'.join(rs)

print(show(grid, {init_state}, init_state))

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


## Movement

In [ ]:
def turn_right(d): return Direction((d.value + 1) % 4)

turn_right(Direction.UP)

<Direction.RIGHT: 1>

In [ ]:
def next_step_adjustment(direction):
    m = {Direction.UP: (-1, 0), Direction.DOWN: (1, 0), Direction.LEFT: (0, -1), Direction.RIGHT: (0, 1)}
    return m[direction]
next_step_adjustment(Direction.UP)

(-1, 0)

In [ ]:
def step(state, grid):
    r, c = state.position
    direction = state.direction
    ir, ic = next_step_adjustment(direction)
    new_r, new_c = r + ir, c + ic

    if 0 <= new_r < len(grid) and 0 <= new_c < len(grid[0]):
        if grid[new_r, new_c]: # Obstacle -> Turn
            new_direction = turn_right(direction)
            return State((r, c), new_direction)
        else: # Continue in the same direction
            return State((new_r, new_c), direction)        
    else:
        return None

init_state, step(init_state, grid)

(State(position=(6, 4), direction=<Direction.UP: 0>),
 State(position=(5, 4), direction=<Direction.UP: 0>))

In [ ]:
from enum import Enum

class End(Enum):
    OUT_OF_MAP = 0
    LOOP = 1

In [ ]:
def run(curr_state, states, grid):
    while True:
        new_state = step(curr_state, grid)
        if not new_state:
            return End.OUT_OF_MAP, states
        elif new_state in states:
            return End.LOOP, states
        else:
            curr_state = new_state
            states.add(new_state)
            
_, states = run(init_state, {init_state}, grid)
len(L(states).map(lambda s: s.position).unique())

41

In [ ]:
def solve(txt):
    xs = txt.splitlines()
    grid = parse_grid(xs)
    init = parse_init_state(xs)
    _, trace = run(init, {init}, grid)
    return len(L(trace).map(lambda s: s.position).unique())

solve(samp)

41

In [ ]:
solve(inp)

5239

# Part 2

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:
```
....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
```
Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
```
Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:
```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
```
Option four, put an alchemical retroencabulator near the bottom left corner:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
```
Option five, put the alchemical retroencabulator a bit to the right instead:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
```
Option six, put a tank of sovereign glue right next to the tank of universal solvent:

```
....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
```
It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?

Can you help me with a plan to solve the part2

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at Part 2, you need to find all positions where adding a single obstruction would cause the guard to loop.

Here's a potential approach:

**Key insight:** You can only place an obstruction where the guard would normally walk (otherwise it won't affect the path at all). Also, you can't place it at the starting position.

**Possible plan:**
1. Get all the positions the guard visits in the original path (you already have this from Part 1)
2. For each of those positions (except the start), try placing an obstruction there
3. Run the simulation with that obstruction and check if it results in a loop
4. Count how many positions cause loops

You already have most of the pieces:
- Your `run()` function already detects loops (returns `End.LOOP`)
- You can modify the grid temporarily to add an obstruction

Does this approach make sense to you? Would you like to start implementing it, or do you have questions about any part of the plan?

In [ ]:
def parse_grid_line(x):
    return L(list(x)).map(lambda c: c in "#O")
parse_grid_line(x)

(#10) [False,True,False,False,False,False,False,False,False,False]

In [ ]:
samp2 = '''....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...'''

grid2 = parse_grid(samp2.splitlines())
init2 = parse_init_state(samp2.splitlines())
run(init2, {init2}, grid2)[0]

<End.LOOP: 1>

In [ ]:
def add_obstacle(pos, grid):
    g = grid.copy()
    g[pos[0]][pos[1]] = True
    return g

print(show(grid, {init_state}, init_state))
print()
print(show(add_obstacle((0,0), grid), {init_state}, init_state))
print()
print(show(grid, {init_state}, init_state))

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...

#...#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


In [ ]:
def is_loop(grid, init_state):
    end, _ = run(init_state, {init_state}, grid)
    return end == End.LOOP
is_loop(grid, init_state), is_loop(grid2, init2)

(False, True)

In [ ]:
def solve2(txt):
    xs = txt.splitlines()
    grid = parse_grid(xs)
    init = parse_init_state(xs)
    _, trace = run(init, {init}, grid)

    obstacle_positions = L(trace).map(lambda s: s.position).filter(lambda p: p != init.position).unique()
    # obstacle_positions = obstacle_positions[:100]
    return obstacle_positions.map(add_obstacle, grid=grid).map(is_loop, init_state=init).sum()

solve2(samp)

6

In [ ]:
# %timeit solve2(inp)

In [ ]:
# solve2(inp)

%timeit solve2(inp) (for 100 samples)

```2.62 s ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)```

total samples: 5238 => will take about 150s

The solve2 function seems to be slow. It works on small data but takes too long on the big input. Can you spot a problem?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your `solve2` function, I can see a potential performance issue. Let me trace through what's happening:

```python
obstacle_positions.map(add_obstacle, grid=grid).map(is_loop, init_state=init)
```

You're creating a new grid for each obstacle position, then checking if it causes a loop. That part looks reasonable.

However, I notice in your `is_loop` function:
```python
def is_loop(grid, init_state):
    end, _ = run(init_state, {init_state}, grid)
    return end == End.LOOP
```

And in your `run` function, you're tracking all states in a set to detect loops. For loops, this could mean visiting thousands of positions multiple times before detecting the cycle.

One potential issue: Are you sure you need to check *all* positions from the original trace? The problem says you need positions where adding an obstacle causes a loop, but maybe there's redundancy?

Actually, wait - I see you have about 5238 positions to check, and each simulation might take a while if it's a long path before looping or exiting. That's the bottleneck.

Do you want to explore optimization strategies, or is 2-3 seconds acceptable for your purposes?

I want to explore optimization strategies.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's think about what makes this slow and how we might speed it up.

The main bottleneck is that you're running ~5238 simulations, and each one potentially walks through many steps before either exiting the map or detecting a loop.

Here are some optimization strategies to consider:

1. **Early loop detection**: Your current approach stores all states and checks if you've seen a state before. This is correct, but the set operations might be slow if states accumulate. Are there ways to make the state comparison faster?

2. **Limit simulation length**: If a simulation runs for longer than some maximum number of steps (like the grid area × 4, since there are 4 directions), you know it must be looping. You could add a step counter and bail out early.

3. **Smarter obstacle placement**: Not every position in the original path is equally likely to cause a loop. Could you filter candidates before testing them all?

4. **Parallel processing**: Since each simulation is independent, you could run multiple checks simultaneously.

Which of these sounds most interesting to explore first, or do you have another idea?

Let's explore the parallel processing

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Parallel processing is a great fit here since each obstacle test is completely independent.

Python has a few options for parallelization. The simplest for CPU-bound tasks like this is the `multiprocessing` module or `concurrent.futures.ProcessPoolExecutor`.

Here's the basic idea - instead of:
```python
obstacle_positions.map(add_obstacle, grid=grid).map(is_loop, init_state=init).sum()
```

You'd use something like:
```python
from concurrent.futures import ProcessPoolExecutor
```

And then map your `is_loop` checks across multiple processes.

However, there's one thing to consider first: your current code passes `grid` and `init_state` to each check. For multiprocessing to work, these need to be picklable (serializable). 

Do you want to try implementing the parallel version yourself, or would you like me to write out a more complete example for you?

Please write it for me

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a parallel version of your `solve2` function:

```python
from concurrent.futures import ProcessPoolExecutor

def check_obstacle_at_position(args):
    pos, grid, init = args
    new_grid = add_obstacle(pos, grid)
    return is_loop(new_grid, init)

def solve2_parallel(txt, max_workers=None):
    xs = txt.splitlines()
    grid = parse_grid(xs)
    init = parse_init_state(xs)
    _, trace = run(init, {init}, grid)

    obstacle_positions = L(trace).map(lambda s: s.position).filter(lambda p: p != init.position).unique()
    
    # Prepare arguments for parallel processing
    args_list = [(pos, grid, init) for pos in obstacle_positions]
    
    # Run in parallel
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(check_obstacle_at_position, args_list))
    
    return sum(results)
```

The key changes:
- Created a helper function `check_obstacle_at_position` that takes all needed arguments as a tuple (required for multiprocessing)
- Used `ProcessPoolExecutor` to distribute the work across multiple CPU cores
- `max_workers=None` uses all available cores by default

Try running `solve2_parallel(samp)` first to verify it works, then test it on `inp` and compare the timing with `%timeit`!

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def check_obstacle_at_position(args):
    pos, grid, init = args
    new_grid = add_obstacle(pos, grid)
    return is_loop(new_grid, init)

def solve2_parallel(txt, max_workers=None):
    xs = txt.splitlines()
    grid = parse_grid(xs)
    init = parse_init_state(xs)
    _, trace = run(init, {init}, grid)

    obstacle_positions = L(trace).map(lambda s: s.position).filter(lambda p: p != init.position).unique()
    
    # Prepare arguments for parallel processing
    args_list = [(pos, grid, init) for pos in obstacle_positions]
    
    # Run in parallel
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(check_obstacle_at_position, args_list))
    
    return sum(results)
solve2_parallel(samp)

6

In [ ]:
solve2_parallel(inp)

1753

In [ ]:
%timeit solve2_parallel(inp)

8.68 s ± 34.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
